In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=564a5f479297ec95fd4220c49d6ebbccaf361175e1ee254b8632fc51df99c374
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [6]:
import numpy as np
import pandas as pd
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [7]:
APP_NAME = 'sampleApp'

spark = SparkSession.builder.appName(APP_NAME).getOrCreate()

In [8]:
from pyspark.sql import SparkSession
APP_NAME = "DataFrames"
SPARK_URL = "local[*]"

spark = SparkSession.builder.appName(APP_NAME).getOrCreate()

taxi_df = pd.read_csv('/content/drive/MyDrive/taxi.csv')
taxi = spark.createDataFrame(taxi_df)

print(taxi)

DataFrame[date: string, hour: bigint, minute: bigint, pickups: double]


In [9]:
collected_taxi = taxi.take(5)
showConsoleProgress = False
print(collected_taxi)

[Row(date='2009-01-01', hour=0, minute=0, pickups=24.0), Row(date='2009-01-01', hour=0, minute=30, pickups=35.0), Row(date='2009-01-01', hour=1, minute=0, pickups=25.0), Row(date='2009-01-01', hour=1, minute=30, pickups=25.0), Row(date='2009-01-01', hour=2, minute=0, pickups=16.0)]


In [10]:
from pyspark.sql import SparkSession

APP_NAME = "DataFrames"
SPARK_URL = "local[*]"

spark = SparkSession.builder.appName(APP_NAME).getOrCreate()
showConsoleProgress = False

taxi = spark.read.load('/content/drive/MyDrive/taxi.csv',
                       format='csv', header='true', inferSchema='true')

taxi = taxi.fillna(0)

print(taxi.summary().show())

+-------+------------------+------------------+------------------+
|summary|              hour|            minute|           pickups|
+-------+------------------+------------------+------------------+
|  count|            128974|            128974|            128974|
|   mean|11.566509529052366|15.004419495402175| 29.00832725975778|
| stddev| 6.908556452594711|15.000057500526209|22.449669931429067|
|    min|                 0|                 0|               0.0|
|    25%|                 6|                 0|              11.0|
|    50%|                12|                30|              27.0|
|    75%|                18|                30|              40.0|
|    max|                23|                30|             310.0|
+-------+------------------+------------------+------------------+

None


In [11]:
print(taxi.groupBy("date").mean().select("date", "avg(pickups)").show())

+----------+------------------+
|      date|      avg(pickups)|
+----------+------------------+
|2009-07-25|             18.25|
|2009-11-22|20.020833333333332|
|2010-08-11|29.583333333333332|
|2012-04-17| 30.67391304347826|
|2012-10-06|           30.1875|
|2013-01-22| 47.53191489361702|
|2013-03-26|32.541666666666664|
|2013-05-21|29.319148936170212|
|2013-09-09|           34.9375|
|2014-09-26|              35.5|
|2014-11-12|36.234042553191486|
|2015-03-09|26.479166666666668|
|2015-05-19|27.043478260869566|
|2016-03-01|25.416666666666668|
|2009-06-28|           18.4375|
|2009-08-09|16.104166666666668|
|2009-09-27|20.479166666666668|
|2009-10-19| 29.78723404255319|
|2010-09-24|              43.0|
|2011-01-29|             29.75|
+----------+------------------+
only showing top 20 rows

None


In [12]:
print(taxi.groupBy("date").mean().select("date", "avg(pickups)").sort("avg(pickups)", ascending=False).show())

+----------+------------------+
|      date|      avg(pickups)|
+----------+------------------+
|2012-03-07| 84.41666666666667|
|2011-03-02|             82.75|
|2012-03-09| 71.52083333333333|
|2014-03-05| 69.02083333333333|
|2012-03-10| 66.41666666666667|
|2012-03-22|             64.25|
|2013-03-22|            64.125|
|2013-11-01| 64.02083333333333|
|2013-03-06|62.723404255319146|
|2012-03-08|              62.0|
|2014-03-20|            61.625|
|2011-03-17| 60.97872340425532|
|2011-03-05|60.270833333333336|
|2011-03-04|           58.4375|
|2011-03-06|              58.0|
|2009-03-04|56.645833333333336|
|2011-05-04|             55.75|
|2014-03-06| 54.97872340425532|
|2015-03-04|54.854166666666664|
|2012-03-23|54.645833333333336|
+----------+------------------+
only showing top 20 rows

None


In [17]:
print(spark.sql('SELECT date, AVG(pickups) FROM taxi ' 'GROUP BY date ORDER BY AVG(pickups) DESC').show())

+----------+------------------+
|      date|      avg(pickups)|
+----------+------------------+
|2012-03-07| 84.41666666666667|
|2011-03-02|             82.75|
|2012-03-09| 71.52083333333333|
|2014-03-05| 69.02083333333333|
|2012-03-10| 66.41666666666667|
|2012-03-22|             64.25|
|2013-03-22|            64.125|
|2013-11-01| 64.02083333333333|
|2013-03-06|62.723404255319146|
|2012-03-08|              62.0|
|2014-03-20|            61.625|
|2011-03-17| 60.97872340425532|
|2011-03-05|60.270833333333336|
|2011-03-04|           58.4375|
|2011-03-06|              58.0|
|2009-03-04|56.645833333333336|
|2011-05-04|             55.75|
|2014-03-06| 54.97872340425532|
|2015-03-04|54.854166666666664|
|2012-03-23|54.645833333333336|
+----------+------------------+
only showing top 20 rows

None


In [16]:
from pyspark.sql import SparkSession

APP_NAME = "DataFrames"
SPARK_URL = "local[*]"

spark = SparkSession.builder.appName(APP_NAME).getOrCreate()
showConsoleProgress = False

taxi = spark.read.load('/content/drive/MyDrive/taxi.csv',
                       format='csv', header='true', inferSchema='true')

taxi = taxi.fillna(0)

taxi.registerTempTable("taxi")

print(taxi.summary().show())
print(taxi.groupby("date").mean().select("date", "avg(pickups)").show())
print(taxi.groupby("date").mean().select("date", "avg(pickups)") \
      .sort("avg(pickups)", ascending=False).show())
print(spark.sql('SELECT date, AVG(pickups) FROM taxi '
                'GROUP BY date ORDER BY AVG(pickups) DESC').show())

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


+-------+------------------+------------------+------------------+
|summary|              hour|            minute|           pickups|
+-------+------------------+------------------+------------------+
|  count|            128974|            128974|            128974|
|   mean|11.566509529052366|15.004419495402175| 29.00832725975778|
| stddev| 6.908556452594711|15.000057500526209|22.449669931429067|
|    min|                 0|                 0|               0.0|
|    25%|                 6|                 0|              11.0|
|    50%|                12|                30|              27.0|
|    75%|                18|                30|              40.0|
|    max|                23|                30|             310.0|
+-------+------------------+------------------+------------------+

None
+----------+------------------+
|      date|      avg(pickups)|
+----------+------------------+
|2009-07-25|             18.25|
|2009-11-22|20.020833333333332|
|2010-08-11|29.583333333333332